In [1]:
import tensorflow as tf
from tensorflow.keras import datasets
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds

Some common arguments:

*   `split=`: Which split to read (e.g. `'train'`, `['train', 'test']`, `'train[80%:]'`,...). See our [split API guide](https://www.tensorflow.org/datasets/splits).
*   `shuffle_files=`: Control whether to shuffle the files between each epoch (TFDS store big datasets in multiple smaller files).
*   `data_dir=`: Location where the dataset is saved (
defaults to `~/tensorflow_datasets/`)
*   `with_info=True`: Returns the `tfds.core.DatasetInfo` containing dataset metadata
*   `download=False`: Disable download

By using as_supervised=True, you can get a tuple (features, label) instead for supervised datasets.

In [2]:
# read/ load data 
(ds_train, ds_test), ds_info = tfds.load(
    "mnist",
    split=["train", "test"],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

In [3]:
physical_devices = tf.config.list_physical_devices("GPU")
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 128

In [4]:
def normalize_image(image , label ):
  image = tf.cast(image ,dtype=tf.float32)/255.0 
  return image ,label

In [5]:
# train dataset 
ds_train=ds_train.map(normalize_image,num_parallel_calls=AUTOTUNE)
ds_train=ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits["train"].num_examples)
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(AUTOTUNE)


In [6]:
# Setup for test Dataset
ds_test = ds_test.map(normalize_image, num_parallel_calls=AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.prefetch(AUTOTUNE)

In [7]:
model = keras.Sequential(
    [
        keras.Input((28, 28, 1)),
        layers.Conv2D(32, 3, activation="relu"),
        layers.Flatten(),
        layers.Dense(10, activation="softmax"),
    ]
)

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 flatten (Flatten)           (None, 21632)             0         
                                                                 
 dense (Dense)               (None, 10)                216330    
                                                                 
Total params: 216,650
Trainable params: 216,650
Non-trainable params: 0
_________________________________________________________________


In [9]:
epochs =2
adams = tf.keras.optimizers.Adam()
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
acc_metric = keras.metrics.SparseCategoricalAccuracy()

In [10]:
# train loop
for epoch in range(epochs):
  print(f"/n Training : {epoch+1}")
  for batch_index , (batch_x  , batch_y) in enumerate(ds_train):
    with tf.GradientTape() as tape: 
      y_pred=model(batch_x ,training = True)
      loss=loss_fn(batch_y ,y_pred )

    # find gradient 
    gradient = tape.gradient(loss,model.trainable_weights )
    # update weights 
    adams.apply_gradients(zip(gradient,model.trainable_weights))
    # update accuracy 
    acc_metric.update_state(batch_y , y_pred)

  train_acc = acc_metric.result()
  print(f"Accuracy over epoch {train_acc}")
  acc_metric.reset_states()


/n Training : 1


c:\Users\adity\anaconda3\envs\huggingFace\lib\site-packages\tensorflow\python\util\dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


Accuracy over epoch 0.9213666915893555
/n Training : 2
Accuracy over epoch 0.9726999998092651


In [11]:
# Test Loop
for batch_idx, (x_batch, y_batch) in enumerate(ds_test):
    y_pred = model(x_batch, training=True)
    acc_metric.update_state(y_batch, y_pred)

train_acc = acc_metric.result()
print(f"Accuracy over Test Set: {train_acc}")
acc_metric.reset_states()

Accuracy over Test Set: 0.9779999852180481


# pytorch training loop

In [12]:
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim 
from torch.utils.data import DataLoader
from torchsummary import summary

In [13]:
# load data
mnist =datasets.MNIST(root='data' , download=True)  
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [14]:
# load data 
batch_size =64
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data,batch_size=1000, shuffle=True)

In [15]:
# hyperparmeter
epochs =2
learning_rate =0.01


In [16]:
# model 

class Net(nn.Module):
  def __init__(self):
    super(Net , self).__init__()

    self.conv1 = nn.Conv2d(1 ,10 ,kernel_size=5 )
    self.conv2 = nn.Conv2d(10 ,20 ,kernel_size=5 )
    self.conv2_drop = nn.Dropout2d()
    self.fc1 = nn.Linear(320, 50)
    self.fc2 = nn.Linear(50, 10)
  
  def forward(self ,x ):
    x  = F.relu(F.max_pool2d(self.conv1(x),2))
    x  = F.relu(F.max_pool2d(self.conv2(x),2))
    x = x.view(-1, 320)
    x = F.relu(self.fc1(x))
    x = F.dropout(x, training=self.training)
    x = self.fc2(x)
    return F.log_softmax(x , dim=-1)


model = Net()

# optimizer 
optimizer = optim.Adam(model.parameters() ,lr = learning_rate )

In [17]:
# model summary
summary(model ,(1,28,28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 24, 24]             260
            Conv2d-2             [-1, 20, 8, 8]           5,020
            Linear-3                   [-1, 50]          16,050
            Linear-4                   [-1, 10]             510
Total params: 21,840
Trainable params: 21,840
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.05
Params size (MB): 0.08
Estimated Total Size (MB): 0.14
----------------------------------------------------------------


In [18]:
# define accuracy metric
def accuracy(y_true , y_pred):

    if y_pred.ndim==2:
        y_pred=torch.argmax(y_pred , dim =1)   
    return sum(y_true==y_pred)/y_true.shape[0]

In [19]:
# train loop
def train(epochs):
  model.train()
  
  for _ ,epoch in enumerate(range(epochs)):
    print(f'\n Training {epoch}')
    acc=[]
    for idx , (x_batch , y_batch) in enumerate(train_dataloader):
      optimizer.zero_grad()
      output=model(x_batch)
      loss=torch.nn.CrossEntropyLoss( )(output , y_batch)
      loss.backward()
      optimizer.step()
      acc.append(accuracy(y_batch , output))
    print(f'\n epoch:{epoch} batch {idx} loss :{loss}  accuracy {sum(acc)/len(acc)}')

def test():
  model.eval()
  acc=[]
  loss=0
  for idx , (x_batch , y_batch) in enumerate(test_dataloader):
    output=model(x_batch)
    loss+=torch.nn.CrossEntropyLoss()(output , y_batch)
    acc.append(accuracy(y_batch , output))
  print(f'\n Test loss :{loss/idx}  accuracy ={sum(acc)/len(acc)}')




In [20]:

train(epochs)
test()


 Training 0

 epoch:0 batch 937 loss :0.6078494191169739  accuracy 0.8834788203239441

 Training 1

 epoch:1 batch 937 loss :0.2037341594696045  accuracy 0.9332355856895447

 Test loss :0.09605428576469421  accuracy =0.9745000004768372
